In [2]:
from Crypto.Util.number import bytes_to_long

class RNG:
    def __init__(self, seed):
        self.p = self.next_prime(2**24)  # Số nguyên tố lớn nhất gần 2^24
        print(self.p)  # In giá trị của p
        self.M = self.init_matrix(seed)
        print(self.M)
        self.state = [ord(c) for c in "Mvm"]  # Chuyển "Mvm" thành danh sách các giá trị ASCII
        self.gen = 2  # Giá trị hằng số sinh

    def next_prime(self, n):
        # Tìm số nguyên tố lớn hơn hoặc bằng n
        while True:
            if self.is_prime(n):
                return n
            n += 1

    def is_prime(self, n):
        # Kiểm tra số nguyên tố
        if n < 2:
            return False
        for i in range(2, int(n**0.5) + 1):
            if n % i == 0:
                return False
        return True

    def init_matrix(self, seed):
        # Khởi tạo ma trận 3x3 từ seed
        elements = [bytes_to_long(seed[i:i+3]) for i in range(0, len(seed), 3)]
        if len(elements) < 9:
            elements += [0] * (9 - len(elements))  # Đệm nếu không đủ phần tử
        return [elements[i:i+3] for i in range(0, 9, 3)]  # Chuyển thành ma trận 3x3

    def matrix_vector_mult(self, matrix, vector):
        # Nhân ma trận với vector, tính theo modulo p
        result = [0] * len(vector)
        for i in range(len(matrix)):
            for j in range(len(vector)):
                result[i] += matrix[i][j] * vector[j]
                result[i] %= self.p
        return result

    def get_random_num(self):
        # Nhân ma trận với vector
        out = self.matrix_vector_mult(self.M, self.state)

        # Cập nhật trạng thái vector
        self.state = [pow(self.gen, x, self.p) for x in self.state]

        # Tính tích vô hướng của out và state
        random_num = sum(o * s for o, s in zip(out, self.state)) % self.p
        return random_num

flag = b"MVM{???????????????????????????}"
seed = flag[4:-1]

rng = RNG(seed)
samples = []

for i in range(9):
    samples.append(rng.get_random_num())

print(f"{samples = }")
samples = [6192533, 82371, 86024, 4218430, 12259879, 16442850, 6736271, 7418630, 15483781]


16777259
[[4144959, 4144959, 4144959], [4144959, 4144959, 4144959], [4144959, 4144959, 4144959]]
samples = [11144394, 16776215, 1555054, 3471308, 1510282, 12133261, 829684, 7548780, 13046208]


In [3]:
p = 16777259
s1 = [ord('M')]
s2 = [ord('v')]
s3 = [ord('m')]
for i in range(9):
    s1.append(pow(2, s1[-1], p))
    s2.append(pow(2, s2[-1], p))
    s3.append(pow(2, s3[-1], p))
    print(s1[-2] * s1[-1] % p)
    print(s2[-2] * s1[-1] % p)
    print(s3[-2] * s1[-1] % p)
    print(s1[-2] * s2[-1] % p)
    print(s2[-2] * s2[-1] % p)
    print(s3[-2] * s2[-1] % p)
    print(s1[-2] * s3[-1] % p)
    print(s2[-2] * s3[-1] % p)
    print(s3[-2] * s3[-1] % p)
    print()
# print(s1, s2, s3)

5421860
1772230
7892987
9638558
172383
7979144
9652642
3244377
3423465

12602992
3795781
14195994
763782
10770925
14471762
7120087
15164489
5436352

81280
13297582
5188020
12055128
9176643
6266445
10876136
6276973
8359708

304775
1253539
15046576
10549195
4338032
11214501
13807517
3539208
12909813

13328106
596369
14295719
5839735
6352823
1522833
15609058
2484827
13077769

13412753
965895
2457462
8018079
8456106
2764409
14179279
12210542
7214269

16011651
13995228
11611561
3289986
1416773
1606418
5462611
4583010
13776348

4002279
2284000
13576122
12527367
4581947
6350058
9741066
12536728
1904479

11990063
7725051
4557349
3919137
7533705
16551403
2964158
3316603
16502661



In [5]:
from sympy import Matrix
from Crypto.Util.number import *
def solve_gfp_system(coefficients, results, p):
    # Convert coefficients and results into matrices
    A = Matrix(coefficients)
    b = Matrix(results)

    # Apply modulo p to all elements in matrices
    A_mod = A.applyfunc(lambda x: x % p)
    b_mod = b.applyfunc(lambda x: x % p)

    # Solve the system of equations in GF(p)
    solution = A_mod.inv_mod(p) * b_mod

    # Convert solution to integers for readability
    solution = solution.applyfunc(lambda x: int(x))
    return solution.tolist()

# Define the prime number p
p = 16777259

# Coefficients of the linear equations
coefficients = [
    [5421860, 1772230, 7892987, 9638558, 172383, 7979144, 9652642, 3244377, 3423465],
    [12602992, 3795781, 14195994, 763782, 10770925, 14471762, 7120087, 15164489, 5436352],
    [81280, 13297582, 5188020, 12055128, 9176643, 6266445, 10876136, 6276973, 8359708],
    [304775, 1253539, 15046576, 10549195, 4338032, 11214501, 13807517, 3539208, 12909813],
    [13328106, 596369, 14295719, 5839735, 6352823, 1522833, 15609058, 2484827, 13077769],
    [13412753, 965895, 2457462, 8018079, 8456106, 2764409, 14179279, 12210542, 7214269],
    [16011651, 13995228, 11611561, 3289986, 1416773, 1606418, 5462611, 4583010, 13776348],
    [4002279, 2284000, 13576122, 12527367, 4581947, 6350058, 9741066, 12536728, 1904479],
    [11990063, 7725051, 4557349, 3919137, 7533705, 16551403, 2964158, 3316603, 16502661]
]

# Results of the equations
results = [6192533, 82371, 86024, 4218430, 12259879, 16442850, 6736271, 7418630, 15483781]

# Solve the system
solution = solve_gfp_system(coefficients, results, p)
print('MVM{', end='')
for row in solution:  # Duyệt qua từng hàng (danh sách con)
    for num in row:  # Duyệt qua từng phần tử trong hàng
        print(long_to_bytes(num % p).decode(), end='')  # Tính modulo và in ra kết quả
print('}', end='')

MVM{l1n34r_fuNcT10n5_4r3_my_f4v}